# Sentiment Analysis with OpenAI/GPT
In this notebook, we're going to learn how to analyse the sentiment of book reviews with OpenAI.

In [109]:
import openai
import json
import pandas as pd

In [110]:
reviews_df = pd.read_csv("reviews.csv")
reviews = reviews_df['review'].tolist()
reviews

["Possibly the worst book I've ever read.It's a huge collection of biases for all the possible countries and cultures. The whole book is structured with examples like: if you are working with Chinese people, you should take this approach, instead if your team is composed by German people you should do this etc....",
 'A book full of oversimplifications, generalisations and self-contradiction. Plus many of the examples felt simply made up. Although it had one or two good ideas thrown in there, I am honestly not sure if this book can hardly help anyone.',
 'I had it on my recommendations list for a long time, but my impression was always like: "damn, I don\'t need a book on cultural differences; I\'ve worked in many international enterprises, I have been trained, I have practical experience - it would be just a waste of time". In the end, it wasn\'t (a waste of time).',
 'Candidate for the best book I have read in 2016 unless another one can beat it. The author made is fun to read with g

In [120]:
def analyse_reviews(user_input):
    prompt = f"""
    {user_input}
    Analyse the sentiment of the reviews above and return a JSON array as the result.
Provide sentiment on a scale of 1-100?
The JSON must have these fields: sentiment, sentiment_score.
    """
    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful review analysis tool."},
            {"role": "user", "content": prompt},
        ]
    )
    try:
        generated_text = completion.choices[0].message.content
        return json.loads(generated_text)
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [112]:
analyse_reviews(reviews)

{'sentiment': 'mixed', 'sentiment_score': 64.2}

In [113]:
analyse_reviews(reviews)

[{'sentiment': 'negative', 'sentiment_score': 15},
 {'sentiment': 'negative', 'sentiment_score': 25},
 {'sentiment': 'positive', 'sentiment_score': 70},
 {'sentiment': 'positive', 'sentiment_score': 85},
 {'sentiment': 'positive', 'sentiment_score': 90},
 {'sentiment': 'neutral', 'sentiment_score': 50}]

In [125]:
analyse_reviews(reviews)

{'reviews': [{'sentiment': 'negative', 'sentiment_score': 12},
  {'sentiment': 'negative', 'sentiment_score': 22},
  {'sentiment': 'neutral', 'sentiment_score': 50},
  {'sentiment': 'positive', 'sentiment_score': 85},
  {'sentiment': 'positive', 'sentiment_score': 70},
  {'sentiment': 'neutral', 'sentiment_score': 65}]}

In [115]:
def analyse_reviews(user_input):
    prompt = f"""
    {user_input}
    Analyse the sentiment of the reviews above and return a JSON array as the result.
Provide sentiment on a scale of 1-100?
The JSON must have these fields: sentiment, sentiment_score.
    """
    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful review analysis tool."},
            {"role": "user", "content": prompt},
        ],
        functions=[{"name": "dummy_fn_set_sentiment", "parameters": {
          "type": "object",
          "properties": {
            "sentiments": {
              "type": "array",
              "items": {
                "type": "object",
                "properties": {
                  "sentiment": {"type": "string", "description": "Sentiment of the review"},
                  "sentiment_score": {"type": "integer","description": "Score between 1-100 of the sentiment"}
                }
              }
            }
          }
        }}],
    )
    try:
        generated_text = completion.choices[0].message.function_call.arguments
        return json.loads(generated_text)
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [118]:
sentiments = analyse_reviews(reviews)
sentiments

{'sentiments': [{'sentiment': 'negative', 'sentiment_score': 30},
  {'sentiment': 'negative', 'sentiment_score': 25},
  {'sentiment': 'positive', 'sentiment_score': 70},
  {'sentiment': 'positive', 'sentiment_score': 90},
  {'sentiment': 'positive', 'sentiment_score': 80},
  {'sentiment': 'neutral', 'sentiment_score': 50}]}

In [119]:
sentiment_df = pd.DataFrame(sentiments["sentiments"])
result = pd.concat([reviews_df, sentiment_df], axis=1)
pd.set_option('max_colwidth', 100)
result

,review,sentiment,sentiment_score
0,Possibly the worst book I've ever read.It's a huge collection of biases for all the possible cou...,negative,30
1,"A book full of oversimplifications, generalisations and self-contradiction. Plus many of the exa...",negative,25
2,"I had it on my recommendations list for a long time, but my impression was always like: ""damn, I...",positive,70
3,Candidate for the best book I have read in 2016 unless another one can beat it. The author made ...,positive,90
4,A practical and comprehensive guide to how different cultures should be approached regarding bus...,positive,80
5,The book was OK. It offers a good overview of differences between cultures. Sometimes we may ass...,neutral,50


In [126]:
result.to_csv("reviews_sentiment.csv", index=False)